In [11]:
from sympy.parsing.sympy_parser import parse_expr

In [19]:
parse_expr("0.3x + 3y", evaluate=True, local_dict={"x": 10, "y": 3, "z": 5}, transformations="all")

12

In [46]:
# from sympy import Abs
from sympy import *
from sympy.abc import x

parse_expr("max(1, 2, 3)", local_dict={"x": -10}, evaluate=True)

3

In [1]:
from numericals import Numerical

numerical = Numerical()

numerical.summarize_historical_data("AAPL")

[*********************100%***********************]  1 of 1 completed


{'period': '100d',
 'interval': '1d',
 'time_zone': None,
 'average_open': 149.7589005279541,
 'max_open': 170.97999572753906,
 'min_open': 126.01000213623047,
 'stdev_open': 12.544231347641734}

In [2]:
import re

my_str = "what(the,fuck, is) what(going, to, narnia)"
what = re.findall(r"what\(\w+,\s*\w+,\s*\w+\)",my_str )
what

['what(the,fuck, is)', 'what(going, to, narnia)']

In [3]:
import os

os.environ['stock_symbol_key'] = "fb6844dd-04c4-4a1e-9596-6b451a0461b3"
print('stock_symbol_key' in os.environ)

True
